# V60 Coffee Brew Data Processing | v01
### _Maybe Your First Jupyter Notebook_ | by Dr. Louis Goldford (2024)$^{1}$

This Jupyter Notebook processes coffee brew data from an Acaia Pearl coffee scale using its CSV file. The notebook **outputs a resampled text file**, especially formatted for a MaxMSP `<coll>` object, containing the following **columns** (i.e., values printed on each line):
- Index
- Time (in milliseconds)
- Delta T (difference between successive "frames," or time measurements, in milliseconds)
- Total Weight of coffee & water (in grams)
- Flow Rate of the kettle drip (grams/second)

Unlike the Python scripts we have been making in VS Code, in Jupyter we can evaluate individual blocks of code, rather than evaluating the entire script at once (by running the script on the Terminal). We can also mix code with markdown instructions, and we can test blocks of code easily this way. 

To **evaluate** the code or markdown boxes in this notebook, simply click on a box and press `shift` + `enter`.

_But first_... a helpful review of function definitions and calls in Python...

## 1. What is a Function?

Functions are **self contained modules of code** that accomplish a specific task. They usually 'take in' data as input, process it, and 'return' a result as output. We can use a programming language's built-in functions, such as `print()` or `prompt()` to process data, but we can also **define our own functions**. Doing so helps us work more efficiently by allowing us to **consolidate** and **recycle** our code. To make our own function, we must first _define_ it. To utilize that function in our programming, we must _call_ the function we first _defined_.

For example, let's **define** a very simple function:

In [ ]:
def minorThird(note):
    return note + 3;

Here, we are telling Python what our function is and does, but not using it to process anything yet. The symbol `def` tells Python that we are defining a function. `minorThird` is the name we give to our function. The symbol `note` that we put inside the parens is a **parameter** of our function, meaning we are specifying a **placeholder** for one kind of **input data** we will later _pass into_ our function when it is **called** (that is, when the function is _used_). The symbol `return` gives us the **output** of the function. Here, we return the `note` data transposed up by 3 semitones, yielding a pitch interval of a minor third.

Now, to **use** our function inside our program, let's **call** it:

In [ ]:
minorThird(60)

Evaluate the function call above and you will see the returned output here in the Notebook: `63`. 

This, above, is what a **function call** looks like. Think of it like a _phone call_ — the **definition** is like a phone number telling us where the function lives, and the **call** dials the phone number to the correct location. We only need to type the function's name with parens, `minorThird()`, and inside the parens we **pass** input data to the function for processing. The input data can be a _harcoded value_, like we typed here, or we can use _variables_ to pass _dynamically changing_ input. We can reference our function anywhere inside our code and as often as we want, **recycling its functionality** whenever we need it. We can write it on a single line as we've done above, or we can _place a call to our function_ inside a control loop, a conditional, or anywhere else!  

The input **value** we pass here in the function _call_, which in this case is `60`, is known as an **argument** of the function. However, the **placeholder for input data** we made in the function _definition_ above is a **parameter**, which tells Python, "Our function will have an input data point, called `note`. Expect a value here when the function is called." 

Okay, now that we understand what a function is, let's make some functions to process the coffee data...

## 2. Import Required Libraries

In [ ]:
# Verify that Python is working inside your virtual environment once you have installed it. 
# !which is compatible with macOS. !where is compatible with Windows. 

!which python3
!where python

In [ ]:
import pandas as pd
import numpy as np

print("Pandas version:", pd.__version__)
print("Numpy version:", np.__version__)

## 3. Specify User-Defined Input Options

In [ ]:
# Input CSV file path
csv_file_path = '/Users/louisgoldford/Documents/myVirtualEnvironment/161188_brewmaster2_20241116141946_zKm2X0BDUi.csv'

# Output text file name
output_filename = 'coffeeData_maxColl_v01.txt'

# Other parameters
row_index = 0  # Row index to process (each row of the CSV represents a different brew)
resample_interval_ms = 50  # Resampling interval in milliseconds (by default, the brew is recorded every 200 milliseconds, but if we want more precession, we can use resample)

## 4. Load the CSV File, Study the Pandas DataFrame

In [ ]:
# Load the CSV file into a Pandas DataFrame and display it
try:
    df = pd.read_csv(csv_file_path)
    print("DataFrame loaded successfully:")
    display(df)  # Use display() for a cleaner table format in Jupyter
except Exception as e:
    print(f"An error occurred: {e}")

Notice that our **Pandas DataFrame** includes a number of **columns** that we can reference _by name_. The `brew data` column includes a list of total weight measurements as we add water to the coffee grounds. These are regularly sampled over the course of the `total time` duration of the entire brew, measured in seconds. We can use this to **calculate how often the weight measurements were taken** by the scale. It turns out these were sampled somewhere around 200 milliseconds, which isn't too good for musical precision...

 ## 5. Define Functions for Processing Input Coffee Data
 #### Here, we will create our own functions, sometimes calling _other_ functions within our own function definitions...

In [ ]:
# format time in milliseconds for MaxMSP `coll` file
def format_time_milliseconds(seconds):
    return int(seconds * 1000)  # Convert seconds to milliseconds

# resample the data and create a new set of uniformly spaced time_points, weight measurements, and smoothed_flow_rates
def resample_data(time_points, weights, smoothed_flow_rates, interval_ms):
    # Convert the desired interval from milliseconds to seconds
    interval_s = interval_ms / 1000  
    # Generate a new set of evenly spaced time points, starting at 0 and ending slightly after the last time point
    # np.arange() is a NumPy function that generates an array of evenly spaced values within a specified range with a fixed step size
    # np.arange(start, stop, step, dtype=None)
    new_time_points = np.arange(0, time_points[-1] + interval_s, interval_s)
    # Interpolate the weights for the new time points
    # np.interp() creates new data points by linear interpolation between the original time points
    new_weights = np.interp(new_time_points, time_points, weights)
    # Interpolate the smoothed flow rates for the new time points in the same way
    new_flow_rates = np.interp(new_time_points, time_points, smoothed_flow_rates)
    # Return the resampled time points, weights, and flow rates
    return new_time_points, new_weights, new_flow_rates

# calculate delta t (the change in time) in milliseconds
# this is the difference in time between successive measurements
def calculate_delta_t(time_points):
    # The prepend=0 argument adds a 0 at the beginning for alignment with the original array length
    # The np.diff() function returns differences between elements of a NumPy array (like x2dx in bach), used to find rates of change or differences in time.
    delta_t = np.diff(time_points, prepend=0) * 1000  # Convert seconds to milliseconds
    # Set the first entry to 0 because there's no previous measurement for the first time point
    delta_t[0] = 0  
    # Return the array of time differences (delta t) in milliseconds
    return delta_t


## 6. Define the Main Processing Function
#### One big function to do the work, taking a number of arguments.
#### The output_text_file function includes **calls** to our smaller functions, written above.

In [ ]:
def output_text_file(csv_file_path, output_filename, row_index=0, resample_interval_ms=100):
    try: # try allows us to throw an error, if anything does not process correctly, as an exception (see 'except' below)
        df = pd.read_csv(csv_file_path) # Read the CSV file into a Pandas DataFrame
        if row_index < 0 or row_index >= len(df):
            print(f"Invalid row index. There are {len(df)} rows in the file.")
            return
        
        row = df.iloc[row_index]
            # Syntax: df.iloc[row_index, column_index]: used to access a specific row of a DataFrame by its integer index
        total_time = float(row['total time'])  # get total time in seconds from the 'total time' column
        brew_data = row['brew data'] # get brew data from the column called 'brew data'

        # weights = [float(weight) for weight in brew_data.split(';') if weight.strip()]
            
            # if weight.strip() filters out any empty or whitespace-only entries from the list generated by split(';')
            # float(weight) converts each valid, non-empty string from the filtered list into a floating-point number
        weights = []
        for weight in brew_data.split(';'):
            # .split(';') splits the brew_data string into a list of substrings using the semicolon (;) as a delimiter
            if weight.strip():  # Skip empty or whitespace-only strings
                weights.append(float(weight))  # Convert valid strings to float and add to the list (append) 

        num_measurements = len(weights)

        time_points = np.linspace(0, total_time, num_measurements)
            # np.linspace() is a NumPy function that generates an array of evenly spaced numbers over a specified range. 
            # It is particularly useful when you need to divide an interval into a fixed number of steps.
            # Here, we use it to divide the X axis (time) into evenly spaced time points according to the total_time variable.
        flow_rates = np.diff(weights) / np.diff(time_points)
        flow_rates = np.append(flow_rates, 0)  # Match length
        smoothed_flow_rates = np.maximum(flow_rates, 0)
        
        resampled_time_points, resampled_weights, resampled_flow_rates = resample_data(
            time_points, weights, smoothed_flow_rates, resample_interval_ms
        )
            # Here, we are using our resample_data function, defined in the block above.
        delta_t = calculate_delta_t(resampled_time_points)
            # Here, we are calling (using) our calculate_delta_t function, defined in the block above.
        
        # Write the output text file
        with open(output_filename, 'w') as f:
            for i, (time, dt, weight, flow) in enumerate(zip(resampled_time_points, delta_t, resampled_weights, resampled_flow_rates)):
                f.write(f"{i}, {format_time_milliseconds(time)} {int(dt)} {weight:.2f} {flow:.2f};\n")
                    # Here, we are calling (using) our format_time_milliseconds function, defined in the block above.
        
        print(f"Data saved as {output_filename}")
    except Exception as e:
        print(f"An error occurred: {e}")

## 7. Run the Function, Save the Ouput File

In [ ]:
output_text_file(csv_file_path, output_filename, row_index, resample_interval_ms)

$^{1}$This script is protected by CC BY-SA 4.0: Attribution-ShareAlike 4.0 International. Canonical URL: https://creativecommons.org/licenses/by-sa/4.0/